# Загрузка модулей:

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

import sys
sys.path.append('./src')  # Для Docker из ./ (./run.sh)
sys.path.append('../')    # Для Docker из dl_utils (dl_utils/docker/run.sh)

In [ ]:
from dinoal import Dino
from ipy_utils import show_video, path2link
from cvat import dir2unlabeled_tasks, tasks_auto_annottation, tasks2preview, subtask2xml
from cvat_srv import CVATSRV, tqdm

# Индексация файлов для предразметки:

In [ ]:
path = 'path/to/dir'               # Путь до папки
path = 'path/to/file'              # Путь до файла
path = ['list', 'to', 'files']     # Или список путей до файлов

tasks = dir2unlabeled_tasks(path, desc='Индексация ресурсов')

# Загрузка модели и установка параметров предразметки:

In [ ]:
dino = Dino(
    '/home/user/models/groundingdino_swinb_cogcoor.pth',
    box_threshold=0.35,
    text_threshold=0.25,
    prompt2label={
        'UFO': 'ufo',
        'Plane': 'ufo',
        'Bird': 'ufo',
        'Airship': 'ufo',
        'Helicopter': 'ufo',

        'Boat': 'boat',
        'Speedboat': 'boat',
        'Motor boat': 'boat',
        'Sailboat': 'boat',
        'Cruiser': 'boat',
        'Ship': 'boat',
        'Steamboat': 'boat',

        'Object': 'object',
        'Anything': 'object'
    }
)
undefined_label = 'undefined'

dino.device

In [ ]:
annotation_tasks = tasks_auto_annottation(tasks, dino, label=undefined_label)

# Сборка превью:

In [ ]:
show_video(tasks2preview(annotation_tasks, fps=30), size=320)

# Экспорт в CVAT:

In [ ]:
# Соединение с сервером:
cvat_srv = CVATSRV()
print('Соединение с', str(cvat_srv), 'установлено.')

In [ ]:
# Определяем проект для сохранения задач:

# Имя проекта в CVAT:
proj_name = 'test_proj'

labels = set(dino.prompt2label.values()) | {undefined_label}

# Если проект с зажданным именем существует:
if proj_name in cvat_srv.keys():

    # Берём его:
    proj = cvat_srv[proj_name]

    # Убеждаемся, что все необходимые метки в проекте уже есть:
    proj_labels = {label['name'] for label in proj.labels()}
    diff_labels = labels - proj_labels
    if diff_labels:
        raise ValueError(f'В проекте нет следующих меток: {diff_labels}')

    print('Используется имеющийся проект.')

else:
    proj = cvat_srv.new(proj_name, labels=labels)
    print('Создан новый проект.')

In [ ]:
# Сама выгрузка:

xml_file = 'annotations.xml'
for task in tqdm(annotation_tasks, desc='Выгрузка размеченных задач'):

    assert len(task) == 1

    df, file, true_frames = task[0]

    subtask2xml((df, file, true_frames), xml_file)

    if isinstance(file, str):
        name = os.path.basename(os.path.splitext(file)[0])
    else:
        name = os.path.basename(os.path.dirname(file[0]))

    if name in proj.keys():
        raise ValueError(f'В проекте уже есть задача с именем "{name}"!')

    proj.new(name, file, annotation_file=xml_file)